In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = 'Data/Variants/Infected/alpha_delta/*.csv'
dic_files = Load_files.load_files(path, -18, -16)
data = dic_files['대구']
data

,stdDay,incDec,alpha_variants,delta_variants
0,2021-04-12,16.0,4.16,0.32
1,2021-04-13,9.0,2.34,0.18
2,2021-04-14,11.0,2.86,0.22
3,2021-04-15,18.0,4.68,0.36
4,2021-04-16,4.0,1.04,0.08
...,...,...,...,...
121,2021-08-11,66.0,0.66,62.70
122,2021-08-12,39.0,0.39,37.05
123,2021-08-13,57.0,0.57,54.15
124,2021-08-14,82.0,0.82,77.90


In [3]:
learning_rates_list = [1e-3, 1e-4]
patiences_list = [20, 50]
num_layers_list = [1, 2, 4]
batch_sizes_list = [32, 64]
hidden_sizes_list = [8, 16, 32]
dropout_list = [0.25]
criterions_list = [nn.MSELoss(), criterion2, criterion3]

hyperparameter_dict = {}
i = 0

for lr in learning_rates_list:
    for patience in patiences_list:
        for num_layers in num_layers_list:
            for batch_sizes in batch_sizes_list:
                for hidden_size in hidden_sizes_list:    
                    for dropout in dropout_list:
                        for criterion in criterions_list:
                            hyperparameter_dict[i] = [lr, patience, num_layers, batch_sizes, hidden_size, dropout, criterion]
                            i += 1
                        
print(i)

216


In [4]:
input_size = 3
sequence_length = 60
num_epochs = 10000

df = Prepare_df.processing(data, 'stdDay', 'alpha_variants')

x = df.iloc[:, 0:]
y = df.iloc[:,:1]

ms = MinMaxScaler()
ss = StandardScaler()

ss.fit(x)
ms.fit(y)

MinMaxScaler()

In [5]:
best_model = {}
models_list = ['seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 
               'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']

for num_model in range(6):
    rmse_min = np.inf
    for hyperparameter in tqdm(hyperparameter_dict.values(), 
                               desc = f'대구_{models_list[num_model]}', position = 0):
        
        lr = hyperparameter[0]
        patience = hyperparameter[1]
        num_layers = hyperparameter[2]
        batch_size = hyperparameter[3]
        hidden_size = hyperparameter[4]
        dropout = hyperparameter[5]
        criterion = hyperparameter[6]
        #print('-------------------------------------------------------------------------------------')
        #print('lr:', lr, ' patience:', patience, ' num_layers:', num_layers, 
        #      ' batch_size:', batch_size, ' hidden_size:', hidden_size, ' dropout:', dropout)
    
        x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 51, 60, 7, batch_size, 'mtm')
        
        if num_model == 0:
            model = RNN_encoder_decoder(input_size = input_size, 
                                        hidden_size = hidden_size,
                                        num_layers = num_layers, 
                                        dropout = dropout,
                                        device = device).to(device)
            
        elif num_model == 1:
            model = LSTM_encoder_decoder(input_size = input_size, 
                                         hidden_size = hidden_size,
                                         num_layers = num_layers, 
                                         dropout = dropout,
                                         device = device).to(device)
            
        elif num_model == 2:
            model = GRU_encoder_decoder(input_size = input_size, 
                                        hidden_size = hidden_size,
                                        num_layers = num_layers, 
                                        dropout = dropout,
                                        device = device).to(device)
            
        elif num_model == 3:
            model = BiRNN_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

        elif num_model == 4:
            model = BiLSTM_encoder_decoder(input_size = input_size, 
                                              hidden_size = hidden_size,
                                              num_layers = num_layers, 
                                              dropout = dropout,
                                              device = device).to(device)

        elif num_model == 5:
            model = BiGRU_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

        optimizer = Adam(model.parameters(), lr = lr)
    
        loss_list, model, epoch = Trainer.Many_to_Many(train_loader, 
                                                       test_loader, 
                                                       model, 
                                                       criterion, 
                                                       optimizer, 
                                                       num_epochs, 
                                                       patience,
                                                       device)
    
        label_y, predicted, first_label_y, first_predicted = predict_mtm(model, df, x_ss, y_ms, 
                                                                         7, 0.5, device)
    
        if rmse_min > rmse(label_y[-7:], predicted[-7:]):
            rmse_min = rmse(label_y[-7:], predicted[-7:])
            best_model[models_list[num_model]] = [rmse_min, model, loss_list, epoch, lr, patience, 
                                                  num_layers, batch_size, hidden_size, dropout, criterion]

대구_seq2seq_RNN:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_LSTM:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_GRU:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_BiRNN:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_BiLSTM:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_BiGRU:   0%|          | 0/216 [00:00<?, ?it/s]

In [6]:
best_model

{'seq2seq_RNN': [0.012579106379957381,
  RNN_encoder_decoder(
    (RNN_encoder): RNN_encoder(
      (rnn): RNN(3, 32, num_layers=2, batch_first=True, dropout=0.25)
    )
    (RNN_decoder): RNN_decoder(
      (rnn): RNN(3, 32, num_layers=2, batch_first=True, dropout=0.25)
      (linear): Linear(in_features=32, out_features=1, bias=True)
    )
  ),
  [0.20405232161283493,
   0.12049057334661484,
   0.04922392964363098,
   0.06093728542327881,
   0.07452331483364105,
   0.0634731575846672,
   0.04824023321270943,
   0.04380568861961365,
   0.045021308586001396,
   0.052183972671628,
   0.04443157836794853,
   0.04179983772337437,
   0.03854679688811302,
   0.04060346260666847,
   0.03928273916244507,
   0.04083758592605591,
   0.03863580338656902,
   0.03850835748016834,
   0.03871653228998184,
   0.03738589771091938,
   0.03664748556911945,
   0.033334601670503616,
   0.03716491535305977,
   0.03728316351771355,
   0.038376256823539734,
   0.03878181427717209,
   0.036211131140589714,
  

In [7]:
for key, values in best_model.items():
    save_model(values[1].state_dict(), f"model/Infected/alpha/{key}.pth")
    save_hyperparameter(values[3:], f"hyperparameter/Infected/alpha/{key}.pkl")